**GENERAL INSTRUCTIONS**

This code only works for 100 sequences

u can change the values in for loop but make sure ur file name is "**sequence (x)**"  *Here x∈ Z  &  x ∈ (2,∞)*  

upload an empty xl sheet normally 

upload all ur sequence files into a google drive and run the first cell.

Before running 3rd cell go through copy any of uploaded file's path and paste it in the which starts with "Fasta =" 

and in last cell copy the path of xl sheet and paste it over there.

after running all cells download the xl sheet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd

Mounted at /content/drive


In [ ]:

starting_values = []
ending_values = []

In [ ]:
def file_opener(i):
  
  # go through it before u make any changes

  fasta = open('/content/drive/MyDrive/Viral genomics /sequence ({}).fasta'.format(i)) 
  # after pasting the path make sure u add curly brackets inside normal brackets.

  full_genome = []  # list which contains all sequence parts.
  for line in fasta:
    if line[0]=='>':
      sentence = line.split()
      accession = sentence[0].replace('>','')
    else:
      full_genome.append(line.strip())
  genome= ''.join(full_genome) # string
  print('the complete genome is {}'.format(genome))
  print("The length of the given Sequence is {}".format(len(genome)))
  # to read the coding sequences file 

  coding_file = open('/content/drive/MyDrive/Viral genomics /sequence ({}).txt'.format(i))
  coding_location = []
  protien_id = []
  protien_name = []
 # now u can continue editing only in path.



# code for seperating required strings from the whole text document.
  for line in coding_file:
    if line[0]== '>':
      sentence= line.split()
    
      for i in sentence:
        if (i[1]== 'l' and i[2]== 'o' and i[3]== 'c'):
          coding_location.append(i)
        if (i[1]== 'p' and i[2]== 'r' and  i[8] == '=' ):
          protien_name.append(i)
        if (i[1]== 'p' and i[2]== 'r' and i[9]== 'i'):
         protien_id.append(i)
  coding_location.pop(0)   # to remove the first element which is not necessary for now
  protien_name.pop(0)
  protien_id.pop(0)

  
  for i in coding_location:
    s = i.split('=')
    j = s[1].split('.')
    if (j[0][0] == '<' or j[0][0] == '>'): # an error occurs if values are not accurate, to remove that error this if statement helps.
      j[0]= j[0].replace('>','')
      j[0]= j[0].replace('<','')
    starting_values.append(int(j[0]))
    if (j[2][0] == '>' or j[2][0] == '<'):
      j[2]= j[2].replace('>','')
      j[2]= j[2].replace('<','')
    ending_values.append(int(j[2].replace(']','')))

  checking()
 
  nucleotides = list(genome)

  # for creating columns in the fianl dataframe
  column = [ 'Accession_no.']
  for i in range(len(coding_location)+1): 
    column.append('Non-CDS_part{}'.format(i+1))
    column.append('Non-CDS_part{}_sequence'.format(i+1))
    if i <len(coding_location) : 
      column.append('CDS_part{}'.format(i+1))
      column.append('CDS_part{}_sequence'.format(i+1))
      column.append('CDS_part{}_location'.format(i+1))

      # to create a table
  final_data= pd.DataFrame(index={'a'} , columns= column)
  final_data['Accession_no.'] = accession

  
  for i in range(len(coding_location)):
    final_data['CDS_part{}_location'.format(i+1)] = coding_location[i]

  #print('Non-Coding Genome Sequences are :')
  non_cds=[]
  j=0  # for while loop
  for i in range(len(starting_values) + 1):#i will go upto 10 from 0

#this if statement is for last iteration which gives an error if runs normally
    if i == len(starting_values) :
      for j in range(ending_values[i-1],len(nucleotides)):  
        non_cds.append(nucleotides[j])
    
      final_data.loc['a','Non-CDS_part{}'.format(i+1)] = '{} to {}'.format(ending_values[i-1]+1 , len(nucleotides))
      final_data.loc['a','Non-CDS_part{}_sequence'.format(i+1)] = "".join(non_cds)  
    else:
     #print("from {}-".format(j+1))
      dummy_variable = j+1
      while (j < (starting_values[i]-1) ):  
       non_cds.append(nucleotides[j])
       j +=1

      final_data.loc['a','Non-CDS_part{}'.format(i+1)] = '{} to {}'.format(dummy_variable, j )
      final_data.loc['a','Non-CDS_part{}_sequence'.format(i+1)] = "".join(non_cds)
    #print("".join(non_cds))
    #print("-{}".format(j+1))
      j = ending_values[i]
    non_cds.clear()

  cds = []
  for i in range(len(starting_values)): # i = 0 to 9
    for j in range(starting_values[i], ending_values[i]):
      cds.append(nucleotides[j])
    final_data.loc['a','CDS_part{}'.format(i+1)] = protien_id[i] , protien_name[i] 
    final_data.loc['a','CDS_part{}_sequence'.format(i+1)] = "".join(cds)
  
  starting_values.clear()
  ending_values.clear()
  
  return final_data



In [ ]:
def checking():
  print('check whether the values entered are correct.')
  cds= pd.DataFrame(list(zip(starting_values,ending_values)),index= np.arange(1,len(starting_values)+1), columns= ('Starting location', 'Ending location'))
  print(cds)
  
  

In [ ]:
xl_data = pd.DataFrame()
for i in range(2,103):
  print(i-1)
  xl_data = xl_data.append(file_opener(i))


1
the complete genome is ...
The length of the given Sequence is 29815
check whether the values entered are correct.
    Starting location  Ending location
1                 236            13453
2               21533            25348
3               25357            26184
4               26209            26436
5               26487            27155
6               27166            27351
7               27358            27723
8               27720            27851
9               28231            29490
10              29515            29631
2
the complete genome is ...
The length of the given Sequence is 29810
check whether the values entered are correct.
    Starting location  Ending location
1                 230            13447
2               21527            25342
3               25351            26178
4               26203            26430
5               26481            27149
6               27160            27345
7               27352            27717
8               27714    

In [ ]:
xl_data

,Accession_no.,Non-CDS_part1,Non-CDS_part1_sequence,CDS_part1,CDS_part1_sequence,CDS_part1_location,Non-CDS_part2,Non-CDS_part2_sequence,CDS_part2,CDS_part2_sequence,...,CDS_part10,CDS_part10_sequence,CDS_part10_location,Non-CDS_part11,Non-CDS_part11_sequence,CDS_part11,CDS_part11_sequence,CDS_part11_location,Non-CDS_part12,Non-CDS_part12_sequence
a,OL466363.1,1 to 235,ACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACT...,"([protein_id=UER32089.1], [protein=ORF1a)",TGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACT...,[location=236..13453],13454 to 21532,GTGCAGCCCGTCTTACACCGTGCGGCACAGGCACTAGTACTGATGT...,"([protein_id=UER32090.1], [protein=surface)",TGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACT...,...,"([protein_id=UER32098.1], [protein=ORF10)",TGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACT...,[location=29515..29631],29632 to 29815,CAATCTTTAATCAGTGTGTAACATTAGGGAGGACTTGAAAGAGCCA...,NaN,NaN,NaN,NaN,NaN
a,OL466372.1,1 to 229,CAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAA...,"([protein_id=UER32187.1], [protein=ORF1a)",TGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACT...,[location=230..13447],13448 to 21526,GTGCAGCCCGTCTTACACCGTGCGGCACAGGCACTAGTACTGATGT...,"([protein_id=UER32188.1], [protein=surface)",TGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACT...,...,"([protein_id=UER32196.1], [protein=ORF10)",TGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACT...,[location=29509..29625],29626 to 29810,CAATCTTTAATCAGTGTGTAACATTAGGGAGGACTTGAAAGAGCCA...,NaN,NaN,NaN,NaN,NaN
a,OL466377.1,1 to 235,ACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACT...,"([protein_id=UER32242.1], [protein=ORF1a)",TGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACT...,[location=236..13453],13454 to 21532,GTGCAGCCCGTCTTACACCGTGCGGCACAGGCACTAGTACTGATGT...,"([protein_id=UER32243.1], [protein=surface)",TGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACT...,...,"([protein_id=UER32251.1], [protein=ORF10)",TGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACT...,[location=29515..29631],29632 to 29823,CAATCTTTAATCAGTGTGTAACATTGGGGAGGACTTGAAAGAGCCA...,NaN,NaN,NaN,NaN,NaN
a,OL466443.1,1 to 235,ACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACT...,"([protein_id=UER32964.1], [protein=ORF1a)",TGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACT...,[location=236..13453],13454 to 21532,GTGCAGCCCGTCTTACACCGTGCGGCACAGGCACTAGTACTGATGT...,"([protein_id=UER32965.1], [protein=surface)",TGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACT...,...,"([protein_id=UER32973.1], [protein=ORF10)",TGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACT...,[location=29515..29631],29632 to 29823,CAATCTTTAATCAGTGTGTAACATTGGGGAGGACTTGAAAGAGCCA...,NaN,NaN,NaN,NaN,NaN
a,OL466445.1,1 to 235,ACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACT...,"([protein_id=UER32986.1], [protein=ORF1a)",TGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACT...,[location=236..13453],13454 to 21532,GTGCAGCCCGTCTTACACCGTGCGGCACAGGCACTAGTACTGATGT...,"([protein_id=UER32987.1], [protein=surface)",TGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACT...,...,"([protein_id=UER32995.1], [protein=ORF10)",TGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACT...,[location=29515..29631],29632 to 29823,CAATCTTTAATCAGTGTGTAACATTAGGGAGGACTTGAAAGAGCCA...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
a,OL488684.1,1 to 226,CTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCT...,"([protein_id=UEU26612.1], [protein=ORF1a)",TGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACT...,[location=227..13444],13445 to 21523,GTGCAGCCCGTCTTACACCGTGCGGCACAGGCACTAGTACTGATGT...,"([protein_id=UEU26613.1], [protein=surface)",TGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACT...,...,"([protein_id=UEU26621.1], [protein=ORF10)",TGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACT...,[location=29506..29622],29623 to 29767,CAATCTTTAATCAGTGTGTAACATTAGGGAGGACTTGAAAGAGCCA...,NaN,NaN,NaN,NaN,NaN
a,OL488700.1,1 to 130,ATAACTAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCT...,"([protein_id=UEU26787.1], [protein=ORF1a)",TGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACT...,[location=131..13348],13349 to 21427,GTGCAGCCCGTCTTACACCGTGC

In [ ]:
xl_data.to_excel('/content/drive/MyDrive/Viral genomics /OL100_DATA.xlsx')

'All the data was uploaded into the xl sheet and can be downloaded.'

'All the data was uploaded into the xl sheet and can be downloaded.'